In [ ]:
pip install pandas matplotlib seaborn scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score


In [ ]:
# 1. Cargar datos
df = pd.read_csv('winequality-red.csv', sep=';')  # asegúrate del nombre del archivo
print(df.head())

In [ ]:
# 2. Análisis exploratorio
print(df.describe())
print(df.info())

# Correlaciones
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Mapa de calor de correlaciones')
plt.show()

# Distribución de la variable objetivo
sns.countplot(x='quality', data=df)
plt.title('Distribución de la calidad del vino')
plt.show()

In [ ]:
# 3. Preprocesamiento
# No hay valores nulos, se normalizarán variables
X = df.drop('quality', axis=1)
y = df['quality']

# Clasificación binaria para Regresión Logística y Árbol
# 0 = mala calidad (<=5), 1 = buena calidad (>=6)
y_binaria = (y >= 6).astype(int)

# Escalamiento
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 4. División Train/Test
X_train, X_test, y_train_reg, y_test_reg = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
_, _, y_train_cls, y_test_cls = train_test_split(X_scaled, y_binaria, test_size=0.2, random_state=42)

In [ ]:
# 5. Modelos

# Árbol de Decisión
tree = DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(X_train, y_train_cls)
y_pred_tree = tree.predict(X_test)

print("\n[Árbol de Decisión]")
print(confusion_matrix(y_test_cls, y_pred_tree))
print(classification_report(y_test_cls, y_pred_tree))

In [ ]:
# 6. Visualizaciones de resultados

# Regresión Lineal: comparación real vs predicho
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test_reg, y=y_pred_rl)
plt.xlabel('Calidad real')
plt.ylabel('Calidad predicha')
plt.title('Regresión Lineal: Real vs Predicho')
plt.show()

# Árbol de decisión
plt.figure(figsize=(15, 8))
plot_tree(tree, feature_names=df.columns[:-1], class_names=["Mala", "Buena"], filled=True)
plt.title('Árbol de Decisión')
plt.show()

# 7. Importancia de variables
importancias = pd.Series(tree.feature_importances_, index=df.columns[:-1])
importancias.nlargest(10).plot(kind='barh')
plt.title('Importancia de características (Árbol)')
plt.show()